In [7]:
import pandas as pd
from pandas import Series,DataFrame

# numpy, matplotlib, seaborn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline
import graphlab

In [8]:
titanic_df = pd.read_csv("train.csv", dtype={"Age": np.float64}, )
test_df    = pd.read_csv("test.csv", dtype={"Age": np.float64}, )

train=graphlab.SFrame.read_csv('train.csv')
titanic_df,test_df = train.random_split(.8,seed=1)
testing=test_df
titanic_df=titanic_df.to_dataframe()
test_df=test_df.to_dataframe()
test_df.drop("Survived", axis=1, inplace=True)

In [9]:
titanic_df.info()
print "---"
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 90.5+ KB
---
<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null f

In [10]:
#pclass column

pclass_dummies_titanic=pd.get_dummies(titanic_df.Pclass)
pclass_dummies_titanic = pclass_dummies_titanic.rename(columns={1: 'PClass=1',2:'PClass=2',3:'PClass=3'})
pclass_dummies_test=pd.get_dummies(test_df.Pclass)
pclass_dummies_test = pclass_dummies_test.rename(columns={1: 'PClass=1',2:'PClass=2',3:'PClass=3'})
titanic_df=titanic_df.join(pclass_dummies_titanic)
test_df=test_df.join(pclass_dummies_test)

titanic_df.drop(['Pclass'], axis=1, inplace=True)
test_df.drop(['Pclass'], axis=1, inplace=True)

In [11]:
titanic_df.head()

,PassengerId,Survived,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,PClass=1,PClass=2,PClass=3
0,1,0,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S,0,0,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C,1,0,0
2,3,1,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S,0,0,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S,1,0,0
4,5,0,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S,0,0,1


In [12]:
#sex column
sex_dummies_titanic=pd.get_dummies(titanic_df.Sex)
sex_dummies_test=pd.get_dummies(test_df.Sex)
titanic_df.join(sex_dummies_titanic)
test_df.join(sex_dummies_test)
titanic_df.drop(['Sex'],axis=1, inplace=True)
test_df.drop(['Sex'],axis=1, inplace=True)

Age column
titanic_df_survived=titanic_df[titanic_df.Survived==1]
titanic_df_unsurvived=titanic_df[titanic_df.Survived==0]
titanic_df_survived_fareMore26=titanic_df_survived[titanic_df_survived.Fare>26]
mean_titanic_df_survived_fareMore26_age=32.16
sd_titanic_df_survived_fareMore26_age=15.27
count_nan = titanic_df_survived_fareMore26["Age"].isnull().sum()
rand_1 = np.random.normal(mean_titanic_df_survived_fareMore26_age,sd_titanic_df_survived_fareMore26_age,count_nan)
titanic_df_survived_fareMore26["Age"][np.isnan(titanic_df_survived_fareMore26["Age"])] = rand_1


In [13]:
#Age column
#filling the na values of survived passengers and fare>26
mean_titanic_df_survived_fareMore26_age=32.16
sd_titanic_df_survived_fareMore26_age=15.27
count_nan = len(titanic_df["Age"][titanic_df.Fare.apply(lambda x: x>26)&np.isnan(titanic_df["Age"])&(titanic_df.Survived==1)])
rand = np.random.normal(mean_titanic_df_survived_fareMore26_age,sd_titanic_df_survived_fareMore26_age,count_nan)
titanic_df["Age"][titanic_df.Fare.apply(lambda x: x>26)&np.isnan(titanic_df["Age"])&(titanic_df.Survived==1)]=rand

#filling the na values of survived passengers and fare<26
mean_titanic_df_survived_fareLess26_age=24.366
sd_titanic_df_survived_fareLess26_age=13.562
count_nan = len(titanic_df["Age"][titanic_df.Fare.apply(lambda x: x<=26)&np.isnan(titanic_df["Age"])&(titanic_df.Survived==1)])
rand = np.random.normal(mean_titanic_df_survived_fareLess26_age,sd_titanic_df_survived_fareLess26_age,count_nan)
titanic_df["Age"][titanic_df.Fare.apply(lambda x: x<26)&np.isnan(titanic_df["Age"])&(titanic_df.Survived==1)]=rand

#filling the na values of unsurvived passengers and fare>10.5
mean_titanic_df_unsurvived_fareMore10_age=31.89
sd_titanic_df_unsurvived_fareMore10_age=16.23
count_nan = len(titanic_df["Age"][titanic_df.Fare.apply(lambda x: x>10.5)&np.isnan(titanic_df["Age"])&(titanic_df.Survived==0)])
rand = np.random.normal(mean_titanic_df_unsurvived_fareMore10_age,sd_titanic_df_unsurvived_fareMore10_age,count_nan)
titanic_df["Age"][titanic_df.Fare.apply(lambda x: x>10.5)&np.isnan(titanic_df["Age"])&(titanic_df.Survived==0)]=rand

#filling the na values of unsurvived passengers and fare<=10.5
mean_titanic_df_unsurvived_fareLess10_age=29.30
sd_titanic_df_unsurvived_fareLess10_age=16.23
count_nan = len(titanic_df["Age"][titanic_df.Fare.apply(lambda x: x<=10.5)&np.isnan(titanic_df["Age"])&(titanic_df.Survived==0)])
rand = np.random.normal(mean_titanic_df_unsurvived_fareLess10_age,sd_titanic_df_unsurvived_fareLess10_age,count_nan)
titanic_df["Age"][titanic_df.Fare.apply(lambda x: x<10.5)&np.isnan(titanic_df["Age"])&(titanic_df.Survived==0)]=rand


C:\Users\ATX\AppData\Local\Dato\Dato Launcher\lib\site-packages\IPython\kernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ATX\AppData\Local\Dato\Dato Launcher\lib\site-packages\IPython\kernel\__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ATX\AppData\Local\Dato\Dato Launcher\lib\site-packages\IPython\kernel\__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ATX\AppData\Local\Dato\Dato Launcher\lib\site-package

In [14]:
#filling out na values of age in test

mean_titanic_df_survived_fareMore26_age=32.16
sd_titanic_df_survived_fareMore26_age=15.27
count_nan = len(test_df["Age"][test_df.Fare.apply(lambda x: x>26)&np.isnan(test_df["Age"])])
rand = np.random.normal(mean_titanic_df_survived_fareMore26_age,sd_titanic_df_survived_fareMore26_age,count_nan)
test_df["Age"][test_df.Fare.apply(lambda x: x>26)&np.isnan(test_df["Age"])]=rand

#filling the na values of test passengers and fare<26
mean_titanic_df_survived_fareLess26_age=24.366
sd_titanic_df_survived_fareLess26_age=13.562
count_nan = len(test_df["Age"][test_df.Fare.apply(lambda x: x<=26)&np.isnan(test_df["Age"])])
rand = np.random.normal(mean_titanic_df_survived_fareLess26_age,sd_titanic_df_survived_fareLess26_age,count_nan)
test_df["Age"][test_df.Fare.apply(lambda x: x<=26)&np.isnan(test_df["Age"])]=rand

#filling the na values of test passengers and fare>10.5
mean_titanic_df_unsurvived_fareMore10_age=31.89
sd_titanic_df_unsurvived_fareMore10_age=16.23
count_nan = len(test_df["Age"][test_df.Fare.apply(lambda x: x>10.5)&np.isnan(test_df["Age"])])
rand = np.random.normal(mean_titanic_df_unsurvived_fareMore10_age,sd_titanic_df_unsurvived_fareMore10_age,count_nan)
test_df["Age"][test_df.Fare.apply(lambda x: x>10.5)&np.isnan(test_df["Age"])]=rand

#filling the na values of unsurvived passengers and fare<=10.5
mean_titanic_df_unsurvived_fareLess10_age=29.30
sd_titanic_df_unsurvived_fareLess10_age=16.23
count_nan = len(test_df["Age"][test_df.Fare.apply(lambda x: x<=10.5)&np.isnan(test_df["Age"])])
rand = np.random.normal(mean_titanic_df_unsurvived_fareLess10_age,sd_titanic_df_unsurvived_fareLess10_age,count_nan)
test_df["Age"][test_df.Fare.apply(lambda x: x<10.5)&np.isnan(test_df["Age"])]=rand


C:\Users\ATX\AppData\Local\Dato\Dato Launcher\lib\site-packages\IPython\kernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ATX\AppData\Local\Dato\Dato Launcher\lib\site-packages\IPython\kernel\__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ATX\AppData\Local\Dato\Dato Launcher\lib\site-packages\IPython\kernel\__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ATX\AppData\Local\Dato\Dato Launcher\lib\site-package

In [15]:
#drop ticket column
titanic_df.drop(['Ticket'], axis=1, inplace=True)
test_df.drop(['Ticket'], axis=1, inplace=True)

In [16]:
#drop cabin column
titanic_df.drop(['Cabin'], axis=1, inplace=True)
test_df.drop(['Cabin'], axis=1, inplace=True)

In [17]:
#embarked column
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 0 to 417
Data columns (total 10 columns):
PassengerId    418 non-null int64
Name           418 non-null object
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Fare           417 non-null float64
Embarked       418 non-null object
PClass=1       418 non-null float64
PClass=2       418 non-null float64
PClass=3       418 non-null float64
dtypes: float64(5), int64(3), object(2)
memory usage: 35.9+ KB


In [18]:
#null fare 
test_df[pd.isnull(test_df).any(axis=1)]

,PassengerId,Name,Age,SibSp,Parch,Fare,Embarked,PClass=1,PClass=2,PClass=3
152,1044,"Storey, Mr. Thomas",60.5,0,0,NaN,S,0,0,1


In [19]:
test_df["Fare"].fillna(28.28, inplace=True)#median fare over age 60

In [20]:
#NAN embarked column
titanic_df[pd.isnull(titanic_df).any(axis=1)]

,PassengerId,Survived,Name,Age,SibSp,Parch,Fare,Embarked,PClass=1,PClass=2,PClass=3
61,62,1,"Icard, Miss. Amelie",38,0,0,80,NaN,1,0,0
829,830,1,"Stone, Mrs. George Nelson (Martha Evelyn)",62,0,0,80,NaN,1,0,0


In [21]:
titanic_df["Embarked"] = titanic_df["Embarked"].fillna("S")

In [22]:
"""
embarked_dummies_titanic=pd.get_dummies(titanic_df.Embarked)
embarked_dummies_test=pd.get_dummies(test_df.Embarked)
embarked_dummies_titanic.drop(['S'], axis=1, inplace=True)
embarked_dummies_test.drop(['S'], axis=1, inplace=True)
titanic_df=titanic_df.join(embarked_dummies_titanic)
test_df=test_df.join(embarked_dummies_test)
"""
titanic_df.drop(['Embarked'], axis=1, inplace=True)
test_df.drop(['Embarked'], axis=1, inplace=True)

In [23]:
def comparefun(x,y1,y2):
    if x<y1:
        if x>=y2:
            return 1
    return 0
"""
#Manipulating Fare Column
titanic_df['fare<7.91']=titanic_df.Fare.apply(lambda x: comparefun(x,np.float64(7.91),np.float64(0)))
test_df['fare<7.91']=test_df.Fare.apply(lambda x: comparefun(x,np.float64(7.91),np.float64(0)))
titanic_df['7.91<fare<14.45']=titanic_df.Fare.apply(lambda x: comparefun(x,np.float64(14.45),np.float64(7.91)))
test_df['7.91<fare<14.45']=test_df.Fare.apply(lambda x: comparefun(x,np.float64(14.45),np.float64(7.91)))
titanic_df['14.45<fare<31']=titanic_df.Fare.apply(lambda x: comparefun(x,np.float64(31),np.float64(14.45)))
test_df['14.45<fare<31']=test_df.Fare.apply(lambda x: comparefun(x,np.float64(31),np.float64(14.45)))
titanic_df['fare>31']=titanic_df.Fare.apply(lambda x: comparefun(x,np.float64(1000),np.float64(31)))
test_df['fare>31']=test_df.Fare.apply(lambda x: comparefun(x,np.float64(10000),np.float64(31)))
"""


titanic_df['7.91<fare<14.45']=titanic_df.Fare.apply(lambda x: comparefun(x,np.float64(14.45),np.float64(0)))
test_df['7.91<fare<14.45']=test_df.Fare.apply(lambda x: comparefun(x,np.float64(14.45),np.float64(7.91)))
titanic_df['14.45<fare<31']=titanic_df.Fare.apply(lambda x: comparefun(x,np.float64(31),np.float64(14.45)))
test_df['14.45<fare<31']=test_df.Fare.apply(lambda x: comparefun(x,np.float64(31),np.float64(14.45)))
titanic_df['fare>31']=titanic_df.Fare.apply(lambda x: comparefun(x,np.float64(1000),np.float64(31)))
test_df['fare>31']=test_df.Fare.apply(lambda x: comparefun(x,np.float64(10000),np.float64(31)))

titanic_df.drop(['Fare'], axis=1, inplace=True)
test_df.drop(['Fare'], axis=1, inplace=True)

In [24]:
#manipulating Sibsp values

def equalComparison(x,y):
    if x==y:
        return 1
    else: 
        return 0
def equalGreaterComparison(x,y,z):
    if x<y:
        if x>=z:
            return 1
    return 0
"""    
for i in range(8):
    colName="Sibsp="+str(i)
    titanic_df[colName]=titanic_df.SibSp.apply(lambda x: equalComparison(x,np.float64(i)))
    test_df[colName]=test_df.SibSp.apply(lambda x: equalComparison(x,np.float64(i)))
"""
colName="Sibsp<=2"
titanic_df[colName]=titanic_df.SibSp.apply(lambda x: equalGreaterComparison(x,np.float64(3),np.float64(0)))
test_df[colName]=test_df.SibSp.apply(lambda x: equalGreaterComparison(x,np.float64(3),np.float64(0)))

colName="Sibsp>2"
titanic_df[colName]=titanic_df.SibSp.apply(lambda x: equalGreaterComparison(x,np.float64(100),np.float64(3)))
test_df[colName]=test_df.SibSp.apply(lambda x: equalGreaterComparison(x,np.float64(100),np.float64(3)))

#drop Sibsp columns
titanic_df.drop('SibSp',axis=1, inplace=True)
test_df.drop('SibSp',axis=1, inplace=True)


In [25]:
#manipulating age columns

#titanic_df['age<11']=titanic_df.Age.apply(lambda x: comparefun(x,np.float64(11),np.float64(0)))
#test_df['age<11']=test_df.Age.apply(lambda x: comparefun(x,np.float64(11),np.float64(0)))
titanic_df['11<age<20']=titanic_df.Age.apply(lambda x: comparefun(x,np.float64(20),np.float64(11)))
test_df['11<age<20']=test_df.Age.apply(lambda x: comparefun(x,np.float64(20),np.float64(11)))
titanic_df['20<age<31']=titanic_df.Age.apply(lambda x: comparefun(x,np.float64(31),np.float64(20)))
test_df['20<age<31']=test_df.Age.apply(lambda x: comparefun(x,np.float64(31),np.float64(20)))
titanic_df['age>31']=titanic_df.Age.apply(lambda x: comparefun(x,np.float64(1000),np.float64(31)))
test_df['age>31']=test_df.Age.apply(lambda x: comparefun(x,np.float64(10000),np.float64(31)))

"""
#titanic_df['age<11']=titanic_df.Age.apply(lambda x: comparefun(x,np.float64(11),np.float64(0)))
#test_df['age<11']=test_df.Age.apply(lambda x: comparefun(x,np.float64(11),np.float64(0)))
titanic_df['11<age<20']=titanic_df.Age.apply(lambda x: comparefun(x,np.float64(20),np.float64(0)))
test_df['11<age<20']=test_df.Age.apply(lambda x: comparefun(x,np.float64(20),np.float64(0)))
titanic_df['20<age<31']=titanic_df.Age.apply(lambda x: comparefun(x,np.float64(1000),np.float64(20)))
test_df['20<age<31']=test_df.Age.apply(lambda x: comparefun(x,np.float64(1000),np.float64(20)))
#titanic_df['age>31']=titanic_df.Age.apply(lambda x: comparefun(x,np.float64(1000),np.float64(31)))
#test_df['age>31']=test_df.Age.apply(lambda x: comparefun(x,np.float64(10000),np.float64(31)))
"""
titanic_df.drop(['Age'], axis=1, inplace=True)
test_df.drop(['Age'], axis=1, inplace=True)

In [26]:
titanic_df.columns

Index([u'PassengerId', u'Survived', u'Name', u'Parch', u'PClass=1',
       u'PClass=2', u'PClass=3', u'7.91<fare<14.45', u'14.45<fare<31',
       u'fare>31', u'Sibsp<=2', u'Sibsp>2', u'11<age<20', u'20<age<31',
       u'age>31'],
      dtype='object')

#Logistic Regression

In [27]:
#titanic_df.to_csv('titanic_train.csv', index=False)

In [162]:
import graphlab

In [163]:
train=graphlab.SFrame(titanic_df)

In [164]:
test=graphlab.SFrame(test_df)

In [165]:
#train.column_names()[1:]

In [166]:
model = graphlab.logistic_classifier.create(train, target='Survived', features=train.column_names()[3:],l1_penalty=10, verbose=False)

In [167]:
predict=model.predict(test)
(model.coefficients)

name,index,class,value
(intercept),None,1,-0.100818453752
Parch,None,1,0.0877633558903
PClass=1,None,1,0.5105942687
PClass=2,None,1,0.192062806389
PClass=3,None,1,-0.544120980393
7.91<fare<14.45,None,1,-0.452142264705
14.45<fare<31,None,1,0.0413112543266
fare>31,None,1,0.251996740353
Sibsp<=2,None,1,0.0
Sibsp>2,None,1,-1.20593179931


In [168]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": predict
    })
submission.to_csv('titanic_l_reg.csv', index=False)

#Naive Bayes

In [169]:
target='Survived'
train=titanic_df
titanic_df.drop('Name',axis=1,inplace=True)
feature=train.columns
#train=train.astype(np.float64)
feature

Index([u'PassengerId', u'Survived', u'Parch', u'PClass=1', u'PClass=2',
       u'PClass=3', u'7.91<fare<14.45', u'14.45<fare<31', u'fare>31',
       u'Sibsp<=2', u'Sibsp>2', u'11<age<20', u'20<age<31', u'age>31'],
      dtype='object')

In [170]:
test_df.drop('Name',axis=1, inplace=True)
#test_df=test_df.astype(np.float64)

In [171]:
#probability of target function

def l_function(x):
    if x==True:
        return np.float64(1)
    else: 
        return np.float64(0)

def probability_target(train,target):
    number_of_obsevations=len(train)
    total_target=sum(train[target].apply(lambda x: l_function(x==1)))
    probability_target=np.float64(total_target)/number_of_obsevations
    return probability_target
    

In [172]:
def probabilty_feature_target_function(df,feature,target):
    train=df[feature]
    number_of_obsevations=len(train)
    probability_feature_target=dict()
    features=train.columns[2:]
    one=np.float64(1)
    zero=np.float64(0)
    for j in features:
        union_0=zero
        sum_numerator=zero
        sum_denominator=zero
        for i in range (number_of_obsevations):
            if train[target][i]==one:
                sum_denominator=sum_denominator+one
                if train[j][i]!=zero:
                    sum_numerator=sum_numerator+one
                else:
                    union_0=union_0+one
        probability_feature_target[j]=np.float64(sum_numerator+one)/np.float64(sum_denominator+one+one)
    df=pd.DataFrame.from_dict(probability_feature_target,'index')
    df = df.rename(columns={0: 'featureValue'})#print probability_feature_target
    return df


In [173]:
def probability_feature_notarget_function(df,feature,target):
    train=df[feature]
    number_of_obsevations=len(train)
    probability_feature_target=dict()
    features=train.columns[2:]
    for j in features:
        sum_numerator=0
        sum_denominator=0
        one=np.float64(1)
        zero=np.float64(0)
        for i in range (number_of_obsevations):
            if train[target][i]==zero:
                sum_denominator=sum_denominator+one
                if train[j][i]!=zero:
                    sum_numerator=sum_numerator+one
            #union=train[j][i]==1 & train[target][i]==0
            #sum_numerator=sum_numerator+l_function(union)
            #sum_denominator=sum_denominator+l_function(train[target][i]==0)
        probability_feature_target[j]=np.float64(sum_numerator+one)/np.float64(sum_denominator+one+one)
    df=pd.DataFrame.from_dict(probability_feature_target,'index')
    df = df.rename(columns={0: 'featureValue'})#print probability_feature_target
    return df

In [174]:
def probabilty_nofeature_target_function(df,feature,target):
    train=df[feature]
    number_of_obsevations=len(train)
    probability_feature_target=dict()
    features=train.columns[2:]
    one=np.float64(1)
    zero=np.float64(0)
    for j in features:
        union_0=zero
        sum_numerator=zero
        sum_denominator=zero
        for i in range (number_of_obsevations):
            if train[target][i]==one:
                sum_denominator=sum_denominator+one
                if train[j][i]==zero:
                    sum_numerator=sum_numerator+one
                else:
                    union_0=union_0+one
        probability_feature_target[j]=np.float64(sum_numerator+one)/np.float64(sum_denominator+one+one)
    df=pd.DataFrame.from_dict(probability_feature_target,'index')
    df = df.rename(columns={0: 'featureValue'})#print probability_feature_target
    return df


In [175]:
def probabilty_nofeature_notarget_function(df,feature,target):
    train=df[feature]
    number_of_obsevations=len(train)
    probability_feature_target=dict()
    features=train.columns[2:]
    one=np.float64(1)
    zero=np.float64(0)
    for j in features:
        union_0=zero
        sum_numerator=zero
        sum_denominator=zero
        for i in range (number_of_obsevations):
            if train[target][i]==zero:
                sum_denominator=sum_denominator+one
                if train[j][i]==zero:
                    sum_numerator=sum_numerator+one
                else:
                    union_0=union_0+one
        probability_feature_target[j]=np.float64(sum_numerator+one)/np.float64(sum_denominator+one+one)
    df=pd.DataFrame.from_dict(probability_feature_target,'index')
    df = df.rename(columns={0: 'featureValue'})#print probability_feature_target
    return df


def probabilty_nofeature_target_function(df,feature,target):
    train=df[feature]
    number_of_obsevations=len(train)
    probability_feature_target=dict()
    features=train.columns[3:]
    for j in features:
        sum_numerator=0
        sum_denominator=0
        union_0=0
        union_1=0
        for i in range (number_of_obsevations):
            if train[target][i]==np.float64(1):
                sum_denominator=sum_denominator+1
                if train[j][i]==np.float64(0):
                    union_0=union_0+1
                else:
                    union_1=union_1+1
            union= (train[j][i]==0 & train[target][i]==1)
            sum_numerator=sum_numerator+l_function(union)
            #sum_denominator=sum_denominator+l_function(train[target][i]==1)
            #union_0=union_0+l_function(union)
            #union=train[j][i]==1 & train[target][i]==1
            #union_1=union_1+l_function(union)
        probability_feature_target[j]=np.float64(sum_numerator+1)/np.float64(sum_denominator+2)
        print sum_numerator
        print union_0
        print union_1
        print sum_denominator
        print "----"
    df=pd.DataFrame.from_dict(probability_feature_target,'index')
    df = df.rename(columns={0: 'featureValue'})#print probability_feature_target
    return df


In [176]:
def probabilty_nofeature_target_function(df,feature,target):
    train=df[feature]
    number_of_obsevations=len(train)
    probability_feature_target=dict()
    features=train.columns[3:]
    for j in features:
        sum_numerator=0
        sum_denominator=0
        union_0=0
        union_1=0
        for i in range (number_of_obsevations):
            if train[target][i]==np.float64(1):
                sum_denominator=sum_denominator+1
                if train[j][i]==np.float64(0):
                    union_0=union_0+1
                else:
                    union_1=union_1+1
            union= (train[j][i]==0 & train[target][i]==1)
            sum_numerator=sum_numerator+l_function(union)
            #sum_denominator=sum_denominator+l_function(train[target][i]==1)
            #union_0=union_0+l_function(union)
            #union=train[j][i]==1 & train[target][i]==1
            #union_1=union_1+l_function(union)
        probability_feature_target[j]=np.float64(sum_numerator+1)/np.float64(sum_denominator+2)
        print sum_numerator
        print union_0
        print union_1
        print sum_denominator
        print "----"
    df=pd.DataFrame.from_dict(probability_feature_target,'index')
    df = df.rename(columns={0: 'featureValue'})#print probability_feature_target
    return df

In [177]:
def predict_single_input(vector,prob_target,prob_feature_target,prob_feature_notarget):
    one=np.float64(1)
    zero=np.float64(0)
    features=vector.columns[1:]
    vectorIndex=vector.index[0]
    prob_test_target=one
    prob_test_notarget=one
    for feature in features:
        prob_test_feature_target=prob_feature_target['featureValue'][feature]
        prob_test_feature_notarget=prob_feature_notarget['featureValue'][feature]
        if vector[feature][vectorIndex]==zero:
            prob_test_feature_target=one-prob_test_feature_target
            prob_test_feature_notarget=one-prob_test_feature_notarget
        
        prob_test_target=prob_test_target*(prob_test_feature_target)
        prob_test_notarget=prob_test_notarget*(prob_test_feature_notarget)
    prob_test_target=prob_test_target*prob_target
    prob_test_notarget=prob_test_notarget*prob_notarget
    prob_target_test_feature=prob_test_target/(prob_test_target+prob_test_notarget)
    prob_notarget_test_feature=prob_test_notarget/(prob_test_target+prob_test_notarget)
    
    
    if (prob_target_test_feature>prob_notarget_test_feature):
        return 1
    else:
        return 0

In [178]:
prob_target=probability_target(train,target)
prob_notarget=1-prob_target
prob_feature_target=dict(probabilty_feature_target_function(train,feature,target))
prob_feature_notarget=dict(probability_feature_notarget_function(train,feature,target))

In [179]:
test_df.PassengerId[0]

892

In [180]:
submission=Series()
#submission = submission.append(Series(Y_pred, index=store_ids))
for i in range(len(test_df)):
    vector=test_df.iloc[[i]]
    predict=predict_single_input(vector,prob_target,prob_feature_target,prob_feature_notarget)
    submission = submission.append(Series(predict))
submission = pd.DataFrame({ "PassengerId": test_df.PassengerId, "Survived": submission.values})
submission.to_csv('titanic_nv.csv', index=False)

features=vector.columns[1:]
vectorIndex=1
prob_test_target=1
prob_test_notarget=1
prob_target=probability_target(train,target)
prob_notarget=1-prob_target
prob_feature_target=dict(probabilty_feature_target_function(train,feature,target))
prob_feature_notarget=dict(probability_feature_notarget_function(train,feature,target))
for feature in features:
    prob_test_feature_target=prob_feature_target['featureValue'][feature]
    prob_test_feature_notarget=prob_feature_notarget['featureValue'][feature]
    if vector[feature][vectorIndex]==0:
        prob_test_feature_target=np.float64(1)-prob_test_feature_target
        prob_test_feature_notarget=np.float64(1)-prob_test_feature_notarget
    prob_test_target=prob_test_target*(prob_test_feature_target*prob_target)
    prob_test_notarget=prob_test_notarget*(prob_test_feature_notarget*prob_notarget)
prob_target_test_feature=prob_test_target/(prob_test_target+prob_test_notarget)
prob_notarget_test_feature=1-prob_target_test_feature
print prob_target_test_feature